# <span style="color:plum"> BEDtools analysis. </span>

1. Compare overlap between two 5mC files (nanopolish, tombo) and 2 6mA files (smrtlink, tombo).
2. 

In [9]:
import pybedtools
from pybedtools import BedTool
import os
import glob
import pprint
import numpy # need fro p-value stats
import scipy

In [37]:
#First we need to define the base dirs
DIRS ={}
DIRS['BASE1'] = '/home/anjuni/methylation_calling/pacbio'
DIRS['BASE2'] = '/home/anjuni/analysis'
DIRS['BED_INPUT'] = os.path.join(DIRS['BASE1'], 'input', 'sorted_bed_files')
DIRS['GFF_INPUT'] = os.path.join(DIRS['BASE2'], 'output')
DIRS['BED_OUTPUT'] = os.path.join(DIRS['BASE2'], 'bedtools_out')

In [38]:
#Quick chech if directories exist
for value in DIRS.values():
    if not os.path.exists(value):
        print('%s does not exist' % value)

In [57]:
#Make filepaths
bed_file_list = [fn for fn in glob.iglob('%s/*.bed' % DIRS['BED_INPUT'], recursive=True)]
gff_file_list = [fn for fn in glob.iglob('%s/*anno.gff3' % DIRS['GFF_INPUT'], recursive=True)]
te_file_list = [fn for fn in glob.iglob('%s/*.gff' % DIRS['GFF_INPUT'], recursive=True)]

In [58]:
#Check that the list works
print(*bed_file_list, sep='\n')
print(*gff_file_list, sep='\n')
print(*te_file_list, sep='\n')

/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/5mC_hc_nanopolish_sorted.bed
/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/5mC_tombo_sorted.bed
/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/5mC_nanopolish_sorted.bed
/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/5mC_hc_tombo_sorted.bed
/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/6mA_tombo_sorted.bed
/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/6mA_smrtlink_sorted.bed
/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/6mA_hc_tombo_sorted.bed
/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/6mA_prob_smrtlink_sorted.bed
/home/anjuni/analysis/output/Pst_104E_v13_ph_ctg_combined_sorted_anno.gff3
/home/anjuni/analysis/output/Pst_104E_v13_p_ctg_combined_sorted_anno.gff3
/home/anjuni/analysis/output/Pst_104E_v13_h_ctg_combined_sorted_anno.gff3
/home/anjuni/analysis/output/Pst_104E_v13_h_ctg.REPET.sorted.filtered.super

In [43]:
BED = {}
for file in bed_file_list:
    name = (file[63:-4])
    bed_file = BedTool(file)
    BED[name] = bed_file

In [44]:
pprint.pprint(BED) # see if it works

{'5mC_hc_nanopolish_sorted': <BedTool(/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/5mC_hc_nanopolish_sorted.bed)>,
 '5mC_hc_tombo_sorted': <BedTool(/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/5mC_hc_tombo_sorted.bed)>,
 '5mC_nanopolish_sorted': <BedTool(/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/5mC_nanopolish_sorted.bed)>,
 '5mC_tombo_sorted': <BedTool(/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/5mC_tombo_sorted.bed)>,
 '6mA_hc_tombo_sorted': <BedTool(/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/6mA_hc_tombo_sorted.bed)>,
 '6mA_prob_smrtlink_sorted': <BedTool(/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/6mA_prob_smrtlink_sorted.bed)>,
 '6mA_smrtlink_sorted': <BedTool(/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/6mA_smrtlink_sorted.bed)>,
 '6mA_tombo_sorted': <BedTool(/home/anjuni/methylation_calling/pacbio/input/sorted_bed_files/6mA_tombo_sorted.bed)>}


In [55]:
GFF = {}
for file in gff_file_list:
    name = (file[42:-5])
    gff_file = BedTool(file)
    GFF[name] = gff_file

In [56]:
pprint.pprint(GFF) # see if it works

{'h_ctg_combined_sorted_anno': <BedTool(/home/anjuni/analysis/output/Pst_104E_v13_h_ctg_combined_sorted_anno.gff3)>,
 'p_ctg_combined_sorted_anno': <BedTool(/home/anjuni/analysis/output/Pst_104E_v13_p_ctg_combined_sorted_anno.gff3)>,
 'ph_ctg_combined_sorted_anno': <BedTool(/home/anjuni/analysis/output/Pst_104E_v13_ph_ctg_combined_sorted_anno.gff3)>}


In [61]:
TE = {}
for file in te_file_list:
    name = (file[42:-4])
    te_file = BedTool(file)
    TE[name] = te_file

In [62]:
pprint.pprint(TE) # see if it works

{'h_ctg.REPET.sorted.filtered.superfamily': <BedTool(/home/anjuni/analysis/output/Pst_104E_v13_h_ctg.REPET.sorted.filtered.superfamily.gff)>,
 'p_ctg.REPET.sorted.filtered.superfamily': <BedTool(/home/anjuni/analysis/output/Pst_104E_v13_p_ctg.REPET.sorted.filtered.superfamily.gff)>,
 'ph_ctg.REPET.sorted.filtered.superfamily': <BedTool(/home/anjuni/analysis/output/Pst_104E_v13_ph_ctg.REPET.sorted.filtered.superfamily.gff)>}
